In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate, BatchNormalization, ReLU, LeakyReLU, Add
from tensorflow.keras.models import Model


2024-07-01 08:16:43.664753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 08:16:43.664798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 08:16:43.666099: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-01 08:16:43.677048: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Define image dimensions
IMG_HEIGHT = 256
IMG_WIDTH = 4096
CHANNELS = 1

# Paths to the directories containing images
lhsa_kanjur_path = '/Resized_Selected_LG'
computer_font_path = '/Resized_Selected_Comp'

def load_images_from_folder(folder, img_height, img_width, channels):
    images = []
    for filename in sorted(os.listdir(folder)):
        img = load_img(os.path.join(folder, filename), color_mode='grayscale', target_size=(img_height, img_width))
        if img is not None:
            img_array = img_to_array(img)
            images.append(img_array)
    images = np.array(images).astype('float32')
    images = (images - 127.5) / 127.5  # Normalize images to [-1, 1]
    return images

# Load images
lhsa_kanjur_images = load_images_from_folder(lhsa_kanjur_path, IMG_HEIGHT, IMG_WIDTH, CHANNELS)
computer_font_images = load_images_from_folder(computer_font_path, IMG_HEIGHT, IMG_WIDTH, CHANNELS)

# Split the data into training and validation sets
train_lhsa_kanjur, val_lhsa_kanjur, train_computer_font, val_computer_font = train_test_split(
    lhsa_kanjur_images, computer_font_images, test_size=0.2, random_state=42)

def create_dataset(input_images, target_images, batch_size=1):
    dataset = tf.data.Dataset.from_tensor_slices((input_images, target_images))
    dataset = dataset.shuffle(buffer_size=len(input_images))
    dataset = dataset.batch(batch_size)
    return dataset

# Create TensorFlow datasets
batch_size = 1
train_dataset = create_dataset(train_computer_font, train_lhsa_kanjur, batch_size)
val_dataset = create_dataset(val_computer_font, val_lhsa_kanjur, batch_size)


2024-07-01 08:16:58.978963: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22282 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:41:00.0, compute capability: 8.9


In [5]:
# Residual Block
def residual_block(x, filters, kernel_size=(3, 3)):
    r = Conv2D(filters, kernel_size, padding='same')(x)
    r = BatchNormalization()(r)
    r = ReLU()(r)
    r = Conv2D(filters, kernel_size, padding='same')(r)
    r = BatchNormalization()(r)
    r = Add()([r, x])
    return r


In [6]:
# Generator with reduced latent dimension to 4x64
def build_generator(input_shape):
    inputs = Input(input_shape)  # (256, 4096, 1)
    # Encoder
    e1 = Conv2D(64, (4, 4), strides=2, padding='same')(inputs)  # (128, 2048, 64)
    e2 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(128, (4, 4), strides=2, padding='same')(e1)))  # (64, 1024, 128)
    e3 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(256, (4, 4), strides=2, padding='same')(e2)))  # (32, 512, 256)
    e4 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(512, (4, 4), strides=2, padding='same')(e3)))  # (16, 256, 512)
    e5 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(1024, (4, 4), strides=2, padding='same')(e4)))  # (8, 128, 1024)
    e6 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(2048, (4, 4), strides=2, padding='same')(e5)))  # (4, 64, 2048)
    
    # Adding Residual Blocks to the Latent Space
    r = residual_block(e6, 2048)  # (4, 64, 2048)
    r = residual_block(r, 2048)  # (4, 64, 2048)
    
    # Decoder with additional layers and skip connections
    d1 = ReLU()(BatchNormalization()(Conv2DTranspose(1024, (4, 4), strides=2, padding='same')(r)))  # (8, 128, 1024)
    d1 = Concatenate()([d1, e5])  # (8, 128, 2048)
    d1 = residual_block(d1, 2048)  # (8, 128, 2048)
    
    d2 = ReLU()(BatchNormalization()(Conv2DTranspose(512, (4, 4), strides=2, padding='same')(d1)))  # (16, 256, 512)
    d2 = Concatenate()([d2, e4])  # (16, 256, 1024)
    d2 = residual_block(d2, 1024)  # (16, 256, 1024)
    
    d3 = ReLU()(BatchNormalization()(Conv2DTranspose(256, (4, 4), strides=2, padding='same')(d2)))  # (32, 512, 256)
    d3 = Concatenate()([d3, e3])  # (32, 512, 512)
    d3 = residual_block(d3, 512)  # (32, 512, 512)
    
    d4 = ReLU()(BatchNormalization()(Conv2DTranspose(128, (4, 4), strides=2, padding='same')(d3)))  # (64, 1024, 128)
    d4 = Concatenate()([d4, e2])  # (64, 1024, 256)
    d4 = residual_block(d4, 256)  # (64, 1024, 256)
    
    d5 = ReLU()(BatchNormalization()(Conv2DTranspose(64, (4, 4), strides=2, padding='same')(d4)))  # (128, 2048, 64)
    d5 = Concatenate()([d5, e1])  # (128, 2048, 128)
    d5 = residual_block(d5, 128)  # (128, 2048, 128)
    
    d6 = Conv2DTranspose(1, (4, 4), strides=2, padding='same', activation='tanh')(d5)  # (256, 4096, 1)
    
    return Model(inputs, d6)


# Discriminator with added complexity
def build_discriminator(input_shape):
    inputs = Input(input_shape)  # (256, 4096, 1)
    target = Input(input_shape)  # (256, 4096, 1)
    combined = Concatenate()([inputs, target])  # (256, 4096, 2)
    
    d1 = LeakyReLU(alpha=0.2)(Conv2D(64, (4, 4), strides=2, padding='same')(combined))  # (128, 2048, 64)
    d2 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(128, (4, 4), strides=2, padding='same')(d1)))  # (64, 1024, 128)
    d3 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(256, (4, 4), strides=2, padding='same')(d2)))  # (32, 512, 256)
    d4 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(512, (4, 4), strides=1, padding='same')(d3)))  # (32, 512, 512)
    d5 = Conv2D(1, (4, 4), strides=1, padding='same')(d4)  # (32, 512, 1)
    
    return Model([inputs, target], d5)

# Input shape (256, 4096, 1)
input_shape = (256, 4096, 1)
generator = build_generator(input_shape)
discriminator = build_discriminator(input_shape)


In [7]:
# Define the optimizers
generator_optimizer = Adam(2e-4, beta_1=0.5)
discriminator_optimizer = Adam(2e-4, beta_1=0.5)

# Define the losses
mse = MeanSquaredError()
mae = MeanAbsoluteError()


In [8]:
class Pix2Pix(Model):
    def __init__(self, generator, discriminator, lambda_value=50):
        super(Pix2Pix, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.lambda_value = lambda_value
        self.generator_loss_tracker = tf.keras.metrics.Mean(name="generator_loss")
        self.discriminator_loss_tracker = tf.keras.metrics.Mean(name="discriminator_loss")

    def compile(self, generator_optimizer, discriminator_optimizer, gen_loss_fn, disc_loss_fn, **kwargs):
        super(Pix2Pix, self).compile(**kwargs)
        self.generator_optimizer = generator_optimizer
        self.discriminator_optimizer = discriminator_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn

    @property
    def metrics(self):
        return [self.generator_loss_tracker, self.discriminator_loss_tracker]

    def train_step(self, data):
        input_image, target = data

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            fake_image = self.generator(input_image, training=True)

            disc_real_output = self.discriminator([input_image, target], training=True)
            disc_fake_output = self.discriminator([input_image, fake_image], training=True)

            gen_gan_loss = self.gen_loss_fn(tf.ones_like(disc_fake_output), disc_fake_output)
            gen_l1_loss = tf.reduce_mean(tf.abs(target - fake_image))
            gen_total_loss = gen_gan_loss + (self.lambda_value * gen_l1_loss)

            disc_real_loss = self.disc_loss_fn(tf.ones_like(disc_real_output), disc_real_output)
            disc_fake_loss = self.disc_loss_fn(tf.zeros_like(disc_fake_output), disc_fake_output)
            disc_total_loss = disc_real_loss + disc_fake_loss

        generator_gradients = gen_tape.gradient(gen_total_loss, self.generator.trainable_variables)
        discriminator_gradients = disc_tape.gradient(disc_total_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(generator_gradients, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.discriminator.trainable_variables))

        self.generator_loss_tracker.update_state(gen_total_loss)
        self.discriminator_loss_tracker.update_state(disc_total_loss)

        return {"generator_loss": self.generator_loss_tracker.result(), "discriminator_loss": self.discriminator_loss_tracker.result()}

    def test_step(self, data):
        input_image, target = data
        fake_image = self.generator(input_image, training=False)

        disc_real_output = self.discriminator([input_image, target], training=False)
        disc_fake_output = self.discriminator([input_image, fake_image], training=False)

        gen_gan_loss = self.gen_loss_fn(tf.ones_like(disc_fake_output), disc_fake_output)
        gen_l1_loss = tf.reduce_mean(tf.abs(target - fake_image))
        gen_total_loss = gen_gan_loss + (self.lambda_value * gen_l1_loss)

        disc_real_loss = self.disc_loss_fn(tf.ones_like(disc_real_output), disc_real_output)
        disc_fake_loss = self.disc_loss_fn(tf.zeros_like(disc_fake_output), disc_fake_output)
        disc_total_loss = disc_real_loss + disc_fake_loss

        return {"generator_loss": gen_total_loss, "discriminator_loss": disc_total_loss}


In [ ]:
# Instantiate the Pix2Pix model
#5+10+5+15+30+100+30
pix2pix_model = Pix2Pix(generator, discriminator)

# Compile the model
pix2pix_model.compile(
    generator_optimizer=generator_optimizer,
    discriminator_optimizer=discriminator_optimizer,
    gen_loss_fn=mse,
    disc_loss_fn=mse
)

# Set epochs
EPOCHS = 100

# Train the model
history = pix2pix_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)


Epoch 1/100
421/800 [==============>...............] - ETA: 1:01 - generator_loss: 6.6736 - discriminator_loss: 6.4516e-04

In [ ]:
import matplotlib.pyplot as plt

def generate_and_display_images(model, test_input, target, index):
    prediction = model(test_input, training=False)
    plt.figure(figsize=(15, 5))  # Increase the figure size to ensure high quality and images on separate lines
    
    display_list = [test_input[0], target[0], prediction[0]]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']
    
    for i in range(3):
        plt.subplot(3, 1, i + 1)  # Change the layout to 3 rows, 1 column
        plt.title(title[i])
        plt.imshow(display_list[i] * 0.5 + 0.5, cmap='gray')  # Rescale to [0, 1] for visualization
        plt.axis('off')
    plt.show()

def evaluate_and_visualize(model, val_dataset, num_images=1):  # Set default to 1 to print only one set of images
    for i, (input_image, target) in enumerate(val_dataset.take(num_images)):
        generate_and_display_images(model.generator, input_image, target, i)

# Visualize the results on examples from the validation dataset
evaluate_and_visualize(pix2pix_model, val_dataset, num_images=1)  # Set to 1 to print only one set of images
